In [2]:
import gymnasium as gym
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

#grayscale helps converting our frames into grayscale
from gym.wrappers import GrayScaleObservation

# import vectorization wrappers
#framestack to capture multiple frames not just one to gather info about how everything moves and at which direction
from stable_baselines3.common.vec_env import VecFrameStack,DummyVecEnv

#import os for file path management
import os

#import PPO for algos
from stable_baselines3 import PPO

#import base Callback for saving models which saves your progress like checkpoints
from stable_baselines3.common.callbacks import BaseCallback


In [3]:
# 1- creating our base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0')

# 2- simplifying the env (instead of discrete(256) it is simplified to discrete(7))
env = JoypadSpace(env, SIMPLE_MOVEMENT)


# 3- grayscale
env = GrayScaleObservation(env, keep_dim=True)


# 4- wrapper inside the Dummy environment
env = DummyVecEnv([lambda: env])

# 5- stack the frames
env = VecFrameStack(env, 4, channels_order='last')


c:\Users\Dell-\anaconda3\Lib\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment SuperMarioBros-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
c:\Users\Dell-\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [4]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq,save_path,verbose=1):
        super(TrainAndLoggingCallback,self).__init__(verbose)
        self.check_freq=check_freq
        self.save_path= save_path

    #This method is intended to be called at the beginning of the training process. It checks if the save_path exists and creates it if it doesn't
    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path,exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq==0:
            model_path= os.path.join(self.save_path,'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True   

In [5]:
CHECKPOINT_DIR ='./callback/'
LOG_DIR='./logs/'

#setup model saving callback
callback= TrainAndLoggingCallback(check_freq=10000,save_path=CHECKPOINT_DIR)

#this is the ai model started
model = PPO('CnnPolicy',env,verbose=1,tensorboard_log=LOG_DIR,learning_rate=0.000001,n_steps=512)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [6]:
#train the AI and saving models with callback function
model.learn(total_timesteps=10000,callback=callback)

Logging to ./logs/PPO_43


c:\Users\Dell-\anaconda3\Lib\site-packages\gym\utils\passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'numpy.ndarray'>`
  logger.warn(
c:\Users\Dell-\anaconda3\Lib\site-packages\gym\utils\passive_env_checker.py:219: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(
c:\Users\Dell-\anaconda3\Lib\site-packages\gym\utils\passive_env_checker.py:225: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(done, (bool, np.bool8)):


----------------------------
| time/              |     |
|    fps             | 119 |
|    iterations      | 1   |
|    time_elapsed    | 4   |
|    total_timesteps | 512 |
----------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 75            |
|    iterations           | 2             |
|    time_elapsed         | 13            |
|    total_timesteps      | 1024          |
| train/                  |               |
|    approx_kl            | 1.7723767e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.95         |
|    explained_variance   | -0.00036      |
|    learning_rate        | 1e-06         |
|    loss                 | 168           |
|    n_updates            | 10            |
|    policy_gradient_loss | -0.000166     |
|    value_loss           | 463           |
-------------------------------------------


In [7]:
#creats a directory called models to save the final model there
model_path = os.path.join('models','model')
model.save(model_path)

c:\Users\Dell-\anaconda3\Lib\site-packages\stable_baselines3\common\save_util.py:283: UserWarning: Path 'models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")
